# **数据准备和特征工程**

数据对于训练和准备模型至关重要。 在本 Notebook 中，我们将介绍如何将数据加载到 ML.NET 中并确保其格式正确，以便 ML.NET 可以使用它。

在本 Notebook 中，您将学习如何...

- 将数据加载到 ML.NET
- 应用数据转换来帮助 ML.NET 理解数据

## **ML.NET加载数据**

### 什么是 IDataView？

[IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) 是 ML.NET 为训练加载的数据格式。 它是一组接口和组件，可为机器学习和高级分析应用程序提供模式化数据的高效组合处理。 它旨在优雅高效地处理高维数据和大型数据集。

IDataView 具有一般模式支持，因为视图可以有任意数量的列，每列都有关联的名称、索引、数据类型和可选注释。

### **如何创建 IDataView**

您可以使用任何加载数据的方法来创建 IDataView：

- 文本加载器
- LoadFromEnumerable
- 数据库加载器
- LoadFromTextFile

有关更多文档和示例，请参阅[从文件和其他来源加载数据](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/load-data-ml-net)。

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"

Installed Packages Microsoft.ML, 2.0.0-preview.22313.1

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms; 

## **下载或查找数据**
以下代码尝试在几个已知位置找到数据文件，或者它将从已知的 GitHub 位置下载它。

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

#### 从文件加载

[TextLoader](https://docs.microsoft.com/dotnet/api/microsoft.ml.data.textloader?view=ml-dotnet) 可以将结构化文件加载到 IDataView 中。 结构化信息表示为数据的列和行。

IDataView 具有一般模式支持，因为视图可以有任意数量的列，每列都有关联的名称、索引、数据类型和可选注释。 您可以使用 Plain-Old-CLR-Objects (POCO) 或类为您的数据定义架构。

关于 ModelInput 类的一些注意事项。
- `LoadColumn` 属性指定列索引。 这是从文件加载时的必要属性。
- `ColumnName` 属性用于将列的名称设置为属性名称以外的名称。 内存中的对象使用属性名称。 但是，对于数据处理和构建机器学习模型，ML.NET 会使用 ColumnName 属性中提供的值覆盖和引用该属性。

In [1]:
public class ModelInput
{
    [LoadColumn(0)]
    [ColumnName(@"vendor_id")]
    public string Vendor_id { get; set; }

    [LoadColumn(1)]
    [ColumnName(@"rate_code")]
    public float Rate_code { get; set; }

    [LoadColumn(2)]
    [ColumnName(@"passenger_count")]
    public float Passenger_count { get; set; }

    [LoadColumn(3)]
    [ColumnName(@"trip_time_in_secs")]
    public float Trip_time_in_secs { get; set; }

    [LoadColumn(4)]
    [ColumnName(@"trip_distance")]
    public float Trip_distance { get; set; }

    [LoadColumn(5)]
    [ColumnName(@"payment_type")]
    public string Payment_type { get; set; }

    [LoadColumn(6)]
    [ColumnName(@"fare_amount")]
    public float Fare_amount { get; set; }

}

所有 ML.NET 操作都从 [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext) 类开始。 初始化 mlContext 会创建一个新的 ML.NET 环境，该环境可以在模型创建工作流对象之间共享。 它在概念上类似于实体框架中的 DBContext。

In [1]:
//Create MLContext
MLContext mlContext = new MLContext();

根据 ModelInput 类型创建一个 [TextLoader](https://docs.microsoft.com/dotnet/api/microsoft.ml.data.textloader?view=ml-dotnet)。 然后使用文本加载器从数据文件中加载。 至少，加载程序需要被告知文件是否有标题，以及文件使用的分隔符。

您也可以使用直接方法 [LoadFromTextFile](https://docs.microsoft.com/dotnet/api/microsoft.ml.textloadersavercatalog.loadfromtextfile?view=ml-dotnet)。 TextLoader 的优点是它提供了从不同位置的多个文件加载文件的选项。

In [1]:
var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");

// Create TextLoader based on the Model Input type. 
TextLoader textLoader = mlContext.Data.CreateTextLoader<ModelInput>(separatorChar: ',', hasHeader: true);

// Load the data into an IDataView. Load() method can support multiple files. 
// Files must they have the same separator character, header, column names, etc. 
IDataView data = textLoader.Load(trainDataPath);

data.Preview(1); 

#### **在内存集合中加载**

ML.NET 支持从内存集合中加载数据。 这使得使用 C# 从 JSON 或 XML 文件加载变得容易。 了解如何 [使用 C# 反序列化 JSON](https://docs.microsoft.com/dotnet/standard/serialization/system-text-json-how-to?pivots=dotnet-6-0#how-to-read-json-as-net-objects-deserialize) 或使用 [XML 序列化程序](https://docs.microsoft.com/dotnet/api/system.xml.serialization?view=net-6.0) 将这些文件放入内存。

在内存中收集数据后，您可以使用 [LoadFromEnumerable](https://docs.microsoft.com/dotnet/api/microsoft.ml.dataoperationscatalog.loadfromenumerable?view=ml-dotnet) 将其加载到 ML.NET ） 方法。

In [1]:
ModelInput[] inMemoryCollection = new ModelInput[]
{
    new ModelInput
    {
        Vendor_id = "CMT",
        Rate_code = 1,
        Passenger_count = 1,
        Trip_time_in_secs = 1271,
        Trip_distance = 3.8f,
        Payment_type = "CRD",
        Fare_amount = 17.5f,
    },
    new ModelInput
    {
        Vendor_id = "VST",
        // missing Rate_code
        Passenger_count = 1,
        Trip_time_in_secs = 474,
        Trip_distance = 1.5f,
        Payment_type = "CSH",
        Fare_amount = 8, 
    }
};

In [1]:
// Create MLContext
MLContext mlContext = new MLContext();

//Load Data
IDataView data = mlContext.Data.LoadFromEnumerable<ModelInput>(inMemoryCollection);

data.Preview(1);

### **DataFrame 和 IDataView 有什么区别？**

您可能听说过 [DataFrame](https://docs.microsoft.com/dotnet/api/microsoft.data.analysis.dataframe?view=ml-dotnet-preview) 类型。它是另一种用于加载、查看和操作笔记本电脑常用数据的工具。它实现了一个 IDataView，因此可以轻松地将其传递给 ML.NET。

DataFrame 和 IDataView 在某种意义上非常相似，它们都是以表格格式表示数据并为其应用转换的方式。一些关键区别：

- DataFrame 仅支持加载分隔文件。
- DataFrame 在内存上运行，因此受限于 PC 上的内存量。

在对数据样本执行探索性数据分析等任务时，建议使用 DataFrame。查看参考笔记本 [REF-数据处理](https://github.com/dotnet/csharp-notebooks/blob/main/machine-learning/REF-Data%20Processing.ipynb) 中使用 Data Frames 的示例操作数据文件进行训练。

建议使用 IDataView 对较大的数据集进行训练，以及本笔记本中的示例所使用的内容。在这种情况下，较大的数据集被定义为无法放入内存的数据集。

## **数据转换**

ML.NET 支持多种数据转换，可将数据转换为所需格式并帮助您更正数据。 一些常见的操作是操作列、规范化值、替换缺失值、转换值等等。

有关详细信息，请参阅 [数据转换](https://docs.microsoft.com/dotnet/machine-learning/resources/transforms)。

以下是一些常见的转换。

### **分类数据**

One Hot 编码是对包含类别的数据的重要转换。 ML 算法要求数据是数字的，它不知道如何处理表示类别的字符串。 vendor_id 和 payment_type 列是分类的，vendor 可以是“CMD”或“VST”，payment 可以是“CReDit”或“CaSH”。 One Hot  采用传入的字符串值并将它们转换为数字数据。

In [1]:
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(
    new[] 
    { new InputOutputColumnPair(@"vendor_id"), 
    new InputOutputColumnPair(@"payment_type")},
    OneHotEncodingEstimator.OutputKind.Binary); 

让我们在 vendor_id 和 payment_type 上测试上述转换。 结果应该是每个类别的向量值。 对于 Vendor_Id，CMT 变为“000”，VST 变为“001”。 我们将为新转换的类型创建一个新的 ModelInputTransformed 类。

In [1]:
using System.Numerics; 

public class ModelInputTransformed
{
    [LoadColumn(0)]
    [ColumnName(@"vendor_id")]
    public VBuffer<Single> Vendor_id { get; set; }

    [LoadColumn(1)]
    [ColumnName(@"rate_code")]
    public float Rate_code { get; set; }

    [LoadColumn(2)]
    [ColumnName(@"passenger_count")]
    public float Passenger_count { get; set; }

    [LoadColumn(3)]
    [ColumnName(@"trip_time_in_secs")]
    public float Trip_time_in_secs { get; set; }

    [LoadColumn(4)]
    [ColumnName(@"trip_distance")]
    public float Trip_distance { get; set; }

    [LoadColumn(5)]
    [ColumnName(@"payment_type")]
    public VBuffer<Single> Payment_type { get; set; }

    [LoadColumn(6)]
    [ColumnName(@"fare_amount")]
    public float Fare_amount { get; set; }
}

In [1]:
// Run the transform
IDataView transformedData = pipeline.Fit(data).Transform(data);
var convertedData = mlContext.Data.CreateEnumerable<ModelInputTransformed>(transformedData, true);

// One Hot Encoding of two columns 'vendor_id' and 'payment_type'.
Console.WriteLine("Vendor_Id" +"\t" + "Payment_Type"); 
foreach (ModelInputTransformed item in convertedData)
{    
    Console.WriteLine("{0}\t\t{1}", string.Join(" ", item.Vendor_id.DenseValues()),
                    string.Join(" ", item.Payment_type.DenseValues()));
}

0 0 1		0 0 1


### **替换缺失值**

另一种常见的操作是替换缺失值。 这里我们使用默认替换模式，将值替换为其类型的默认值。

In [1]:
pipeline.Append(mlContext.Transforms.ReplaceMissingValues(
    new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), 
    new InputOutputColumnPair(@"passenger_count", @"passenger_count"), 
    new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), 
    new InputOutputColumnPair(@"trip_distance", @"trip_distance") })); 

让我们再运行转换并查看填充的值。 我们缺少第二个虚拟对象的 rate_code。

In [1]:
IDataView transformedData = pipeline.Fit(data).Transform(data);
var convertedData = mlContext.Data.CreateEnumerable<ModelInputTransformed>(transformedData, true);

"Rate_code: " + convertedData.ElementAt(1).Rate_code

Rate_code: 0

现在让我们将所有特征列连接成一个向量列。 许多 ML 训练器期望特征是向量类型，因为将操作应用于向量更有效。

In [1]:
pipeline.Append(mlContext.Transforms.Concatenate(
    @"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

我们现在有一个加载的 IDataView 和管道用于训练。

# **进阶学习**

> [⏩ 下一个学习模块 - 训练 and AutoML](https://github.com/kinfey/csharp-notebooks/blob/main/machine-learning/03-Training%20and%20AutoML.ipynb)  
> [⏪ 上一个学习模块 - Intro to Machine Learning](https://github.com/kinfey/csharp-notebooks/blob/main/machine-learning/01-Intro%20to%20Machine%20Learning.ipynb)  